# Engagement Risk Prediction - EDA
### Project: Online Retail
### Notebook: 01_eda.ipynb

**Goals:** 
- Understand Transaction Data
- Check Data Quality
- Define Event-Level Order Table
- Establish Time Range and Basic Statistics

In [1]:
import pandas as pd
from pathlib import Path

DATA_PATH = Path("../data/raw")
files = list(DATA_PATH.glob("*.xls*"))
files

[PosixPath('../data/raw/online_retail_II.xlsx')]

In [2]:
df = pd.read_excel(files[0])
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [3]:
df.shape

(525461, 8)

In [4]:
df.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')

In [5]:
df["InvoiceDate"].min(), df["InvoiceDate"].max()

(Timestamp('2009-12-01 07:45:00'), Timestamp('2010-12-09 20:01:00'))

In [6]:
df["Customer ID"].nunique()

4383

# Dataset Description:
- The dataset contains 525,461 orders (one order per row) with 8 columns.
- Data ranges from early December 2009 to early December 2010
- There are 4,383 unique customers in the dataset

We define a "User Engagement" event: **One Order = One Engagement Event**

## Data Cleansing & Preparation:

Next, we conduct basic data cleansing and preparation tasks which include: 
- Dropping any rows with no Customer ID
- Limit dataset to only purchases (exclude returns and cancellations)
- Create a revenue column (quantity * price)
- Normalize datatypes for IDs and Time Stamp columns

In [7]:
df_clean = df.copy()

# Drop rows without a customer ID
df_clean = df_clean.dropna(subset=["Customer ID"])

# Keep only positive purchases (exclude returns/cancellations)
df_clean = df_clean[df_clean["Quantity"] > 0]
df_clean = df_clean[df_clean["Price"] > 0]

# Revenue per line
df_clean["Revenue"] = df_clean["Quantity"] * df_clean["Price"]

df_clean.shape

(407664, 9)

In [8]:
# Normalize IDs and TimeStamps 

df_clean["Customer ID"] = df_clean["Customer ID"].astype(int).astype(str)
df_clean["Invoice"] = df_clean["Invoice"].astype(str)
df_clean["InvoiceDate"] = pd.to_datetime(df_clean["InvoiceDate"])

The dataset now contains 407,664 transactions with 9 columns 

## Order-Table Construction:

Next, we create an order-level table that will display transaction data grouped by customers and invoice.

In [9]:
# Create Order-Level Table

orders = (
    df_clean
    .groupby(["Customer ID", "Invoice"])
    .agg(
        order_ts=("InvoiceDate", "min"),
        total_revenue=("Revenue", "sum"),
        total_qty=("Quantity", "sum"),
        n_items=("StockCode", "nunique"),
        n_lines=("StockCode", "size"),
    )
    .reset_index()
)

orders.head(), orders.shape

(  Customer ID Invoice            order_ts  total_revenue  total_qty  n_items  \
 0       12346  491725 2009-12-14 08:34:00           45.0         10        1   
 1       12346  491742 2009-12-14 11:00:00           22.5          5        1   
 2       12346  491744 2009-12-14 11:02:00           22.5          5        1   
 3       12346  492718 2009-12-18 10:47:00           22.5          5        1   
 4       12346  492722 2009-12-18 10:55:00            1.0          1        1   
 
    n_lines  
 0        1  
 1        1  
 2        1  
 3        1  
 4        1  ,
 (19213, 7))

In [10]:
# Sanity Check: Dataset Time Frame Check

orders["order_ts"].min(), orders["order_ts"].max()

(Timestamp('2009-12-01 07:45:00'), Timestamp('2010-12-09 20:01:00'))

In [11]:
# Persist orders table via parquet file (will not be committed due to .gitignore)

out = Path('../data/processed')
out.mkdir(parents = True, exist_ok = True)

orders.to_parquet(out / 'orders.parquet', index = False)

## Distribution of Revenue: 

In [12]:
orders["total_revenue"].describe()

count    19213.000000
mean       459.688923
std        931.129071
min          0.840000
25%        159.380000
50%        303.540000
75%        484.440000
max      44051.600000
Name: total_revenue, dtype: float64

**Median = 303 approx.;
Mean = 460 approx.;
Max = 44,051 approx. (strong right skew)**

## Distribution of Orders by Customer:

In [13]:
orders.groupby("Customer ID").size().describe()

count    4312.000000
mean        4.455705
std         8.170213
min         1.000000
25%         1.000000
50%         2.000000
75%         5.000000
max       205.000000
dtype: float64

**Median = 2 Orders per customer;
75% of customers are less than or equal to 5 orders;
Max amount of orders = 205 (right-skewed)**

## High-Level Observations from Exploratory Data Analysis

### Revenue Distribution

Customer revenue is **highly right-skewed**, with a small number of customers accounting for a disproportionately large share of total revenue.

- Median order revenue is substantially lower than the mean, indicating the presence of a long tail of high-value orders.
- A small subset of orders exhibit extremely large total revenue, consistent with bulk purchases or high-frequency buyers.
- This skew suggests that raw revenue features may dominate models if not handled carefully.

**Implications for modeling:**
- Revenue-based features may benefit from transformations (e.g., log scaling) or careful normalization.
- Evaluating model performance across customer segments (low- vs high-value customers) will be important to ensure robustness.
- Engagement risk should not be inferred solely from monetary value, as many low-spend customers remain active.

---

### Orders per Customer

The distribution of orders per customer is also **heavily skewed**, with most customers placing only a small number of orders.

- The median customer places very few orders, while a small minority of customers place dozens or even hundreds of orders.
- This long-tail behavior is typical of retail and subscription-adjacent businesses, where a core group of highly engaged users coexists with many sporadic or one-time buyers.

**Implications for modeling:**
- Frequency-based features (e.g., number of orders in recent windows) are likely to be strong predictors of short-term engagement.
- Cold-start and low-history users represent a significant portion of the population and may be inherently harder to model.
- Simple heuristics based on lifetime activity may perform poorly, motivating the use of time-aware features such as recency and rolling activity windows.

---

### Overall Takeaway

Although the raw dataset contains relatively few columns, the **temporal structure and behavioral heterogeneity** of customers provide rich signal for modeling engagement risk.

These observations motivate:
- The use of rolling time-window features (7/30/90 days)
- Time-based snapshot modeling rather than static aggregation
- Careful evaluation against simple heuristics to demonstrate the value of learned models
